# Combining Prognostic Models

This section demonstrates how prognostic models can be combined. There are two times in which this is useful: 

1. When combining multiple models of different inter-related systems into one system-of-system model (i.e., [Composite Models](https://nasa.github.io/progpy/api_ref/prog_models/CompositeModel.html)), or
2. Combining multiple models of the same system to be simulated together an aggregated (i.e., [Ensemble Models](https://nasa.github.io/progpy/api_ref/prog_models/EnsembleModel.html)). This is generally done to improve the accuracy of prediction when you have multiple models that each represent part of the behavior or represent a distribution of different behaviors. 

These two methods for combining models are described in the following sections.

## Composite Model

A CompositeModel is a PrognosticsModel that is composed of multiple PrognosticsModels. This is a tool for modeling system-of-systems. i.e., interconnected systems, where the behavior and state of one system effects the state of another system. The composite prognostics models are connected using defined connections between the output or state of one model, and the input of another model. The resulting CompositeModel behaves as a single model.

To illustrate this, we will create a compositemodel of an aircraft's electric powertrain, combining the DCMotor, ESC, and PropellerLoad models.

First we will import the used models, and the CompositeModel class

In [ ]:
from progpy.models import DCMotor, ESC, PropellerLoad
from progpy import CompositeModel

Next we will initiate objects of the individual models that will later create the composite powertrain model.

In [ ]:
m_motor = DCMotor()
m_esc = ESC()
m_load = PropellerLoad()

Next we have to define the connections between the systems. Let's first define the connections from the DCMotor to the propeller load.

In [ ]:
print('motor states: ', m_motor.states)
print('load inputs: ', m_load.inputs)

Each of the states and inputs are described in the model documentation at [DC Motor Docs](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#dc-motor) and [Propeller Docs](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#propellerload)

From reading the documentation we understand that the propeller's velocity is from the motor, so we can define the first connection:

In [ ]:
connections = [
    ('DCMotor.v_rot', 'PropellerLoad.v_rot')
]

The connection above telling the composite model to feed the DCMotor's v_rot into the propeller load's input v_rot.

Next, let's look at the connections the other direction, from the load to the motor.

In [ ]:
print('load states: ', m_load.states)
print('motor inputs: ', m_motor.inputs)

We know here that the load on the motor is from the load, so we can add that connection. 

In [ ]:
connections.append(('PropellerLoad.t_l', 'DCMotor.t_l'))

Now we will repeat the exercise with the DCMotor and ESC.

In [ ]:
print('ESC states: ', m_esc.states)
print('motor inputs: ', m_motor.inputs)
connections.append(('ESC.v_a', 'DCMotor.v_a'))
connections.append(('ESC.v_b', 'DCMotor.v_b'))
connections.append(('ESC.v_c', 'DCMotor.v_c'))

print('motor states: ', m_motor.states)
print('ESC inputs: ', m_esc.inputs)
connections.append(('DCMotor.theta', 'ESC.theta'))

Now we are ready to combine the models. We create a composite model with the defined connections.

In [ ]:
m_powertrain = CompositeModel(
        (m_esc, m_load, m_motor), 
        connections=connections)

The resulting model includes two inputs, ESC voltage (from the battery) and duty (i.e., commanded throttle). These are the only two inputs not connected internally from the original three models. The states are a combination of all the states of every system. Finally, the outputs are a combination of all the outputs from each of the individual systems. 

In [ ]:
print('inputs: ', m_powertrain.inputs)
print('states: ', m_powertrain.states)
print('outputs: ', m_powertrain.outputs)

Frequently users only want a subset of the outputs from the original model. For example, in this case you're unlikely to be measuring the individual voltages from the ESC. Outputs can be specified when creating the composite model. For example:

In [ ]:
m_powertrain = CompositeModel(
        (m_esc, m_load, m_motor), 
        connections=connections,
        outputs={'DCMotor.v_rot', 'DCMotor.theta'})
print('outputs: ', m_powertrain.outputs)

Now the outputs are only DCMotor angle and velocity.

The resulting model can be used in simulation, state estimation, and prediction the same way any other model would be, as demonstrated below:

In [ ]:
load = m_powertrain.InputContainer({
        'ESC.duty': 1, # 100% Throttle
        'ESC.v': 23
    })
def future_loading(t, x=None):
    return load

simulated_results = m_powertrain.simulate_to(2, future_loading, dt=2.5e-5, save_freq=1e-2)
fig = simulated_results.outputs.plot(compact=False, keys=['DCMotor.v_rot'], ylabel='Velocity')
fig = simulated_results.states.plot(keys=['DCMotor.i_b', 'DCMotor.i_c', 'DCMotor.i_a'], ylabel='ESC Currents')

## Ensemble Model